<a href="https://colab.research.google.com/github/MengOonLee/WebScrapy/blob/master/CountryHoliday/CountryHoliday.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
%%bash
pip install --no-cache-dir -qU scrapy selenium

In [26]:
%%writefile CountryHoliday.py
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import wait, expected_conditions
import scrapy
from scrapy import crawler
import time

class CountryHolidaySpider(scrapy.Spider):
    name = "CountryHoliday"
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        options = webdriver.chrome.options.Options()
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--enable-javascript")
        options.add_argument("--disable-cookies")
        options.add_argument("--disable-notifications")
        options.add_argument("--disable-web-security")
        options.add_argument("--incognito")
        self.driver = webdriver.Chrome(options=options)

    def start_requests(self):
        urls = ["https://www.timeanddate.com/holidays/?allcountries"]
        for url in urls:
            request = scrapy.Request(url=url, callback=self.parse_country)
            yield request

    def parse_country(self, response):
        self.driver.get(response.url)

        wait.WebDriverWait(self.driver, timeout=10)\
        .until(expected_conditions.presence_of_element_located(
            (By.XPATH, "//article[@class='category-list']")))

        selector = scrapy.Selector(text=self.driver.page_source)
        for item in selector.css("ul.category-list__list > li"):
            country = item.css("::text").get()
            link = item.css("a::attr(href)").get()

            if link is not None:
                link = "https://www.timeanddate.com" + link + "2024?hol=1"
                yield scrapy.Request(link, callback=self.parse_holiday,
                    meta={"country":country})

    def parse_holiday(self, response):
        self.driver.get(response.url)
        time.sleep(10)
        country = response.meta["country"]
        yield {
            "country":country,
            "link":response.url
        }

process = crawler.CrawlerProcess(
    settings={"FEEDS":{"items.jl":{"format":"jsonlines"}}}
)
process.crawl(CountryHolidaySpider)
process.start()

Overwriting CountryHoliday.py


In [ ]:
%%bash
python CountryHoliday.py

In [ ]:
from selenium import webdriver
from selenium.webdriver.support import wait, expected_conditions
from selenium.webdriver.common.by import By
import scrapy
import time

options = webdriver.chrome.options.Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--enable-javascript")
options.add_argument("--disable-cookies")
options.add_argument("--disable-notifications")
options.add_argument("--disable-web-security")
options.add_argument("--incognito")
driver = webdriver.Chrome(options=options)

url = "https://www.timeanddate.com/holidays/?allcountries"
driver.get(url)

wait.WebDriverWait(driver, timeout=10).until(
    expected_conditions.presence_of_element_located(
    (By.XPATH, "//article[@class='category-list']")))

selector = scrapy.Selector(text=driver.page_source)
# countries = selector.css("ul.category-list__list ::text").getall()
for item in selector.css("ul.category-list__list > li"):
    name = item.css("::text").get()
    link = item.css("a::attr(href)").get() + "2024?hol=9"
    print(name, link)

driver.close()